# Analysis of assignments in the function body

**The main objective of this project is to write a function getFlow() which analyses a body of a function given as an argument and returns an object containing a list of all assignments together with conditions.**

The implementation has several flaws inter alia analysing only if/else phrases.

## Table of contents

<div class="toc">
	<ul class="toc-item">
		<li>
			<span>
				<a href="#Implementation" data-toc-modified-id="Implementation">
					<span class="toc-item-num">1.&nbsp;&nbsp;</span>Implementation
				</a>
			</span>
		</li>
		<li>
			<span>
				<a href="#Examples" data-toc-modified-id="Examples">
					<span class="toc-item-num">2.&nbsp;&nbsp;</span>Examples
				</a>
			</span>
		</li>
	</ul>
</div>

## Implementation

In [1]:
library(stringr)

#class assignment
assignment <- function(obj){
  structure(list(a = obj), class = "assignment")
}

#method print for class assignment
print.assignment <- function(obj){
  print(obj$a, quote=FALSE)
}

#class assignmentList
assignmentList <- function(objs){
  structure(list(assignments = objs), class = c("assignmentList","assignment"))
}

#method print for class assignmentList
print.assignmentList <- function(objs){
  for (i in 1:length(objs$assignments)){
    print(objs$assignments[[i]], quote=FALSE)
  }
}

In [2]:
#make pair for left "{" and right "}"
make_pairs <- function(left_brackets, right_brackets){
  
  bracket_pairs <- c()
  
  for (i in length(left_brackets):1){
    for (j in 1:length(right_brackets)){
      if ((left_brackets[[i]] <= right_brackets[[j]])){
        bracket_pairs <- c(bracket_pairs, left_brackets[i], right_brackets[j])
        left_brackets[i] <- -1
        right_brackets[j] <- -1
        break
      }
    }
  }
  return(bracket_pairs)
}

In [3]:
#check to which intervals belong assignments
to_which_interval_belongs <- function(bracket_pairs, assignments){
  assignments_in_intervals <- list()
  for (i in 1:length(assignments)){
    j = 1
    while (j <= length(bracket_pairs)){
      bracket <- (assignments[i] >= bracket_pairs[j] && assignments[i]<= bracket_pairs[j+1])
      assignments_in_intervals <- c(assignments_in_intervals, bracket)
      j = j+2
    }
  }
  return(assignments_in_intervals)
}

In [4]:
#determine the conditions of assignments and return an object of class assignmentList
check_assigments_conditions <- function(assignments_in_intervals, assignments, ifs, elses, if_elses, f, bracket_pairs){
  
  left_bracket = 1
  left <- bracket_pairs[1:(left_bracket+1)!=(left_bracket+1)]
  assignment_base <- list()

  intervals_per_assignment = (length(assignments_in_intervals)/length(assignments))
  
  #for each assignment
  for (i in 1:length(assignments)){
    conditions <- c()
    
    #and for each interval
    for (j in 1:intervals_per_assignment){
      
      #check if assignment[interval] == TRUE
      if (assignments_in_intervals[intervals_per_assignment*(i-1)+j] == TRUE){

        bracket <- ifelse(((intervals_per_assignment*(i-1)+j) %% intervals_per_assignment == 0), intervals_per_assignment,
                          (intervals_per_assignment*(i-1)+j) %% intervals_per_assignment)
        
        #if left[bracket[assignment]] is in ifs
        if (left[bracket] %in% ifs) {
          
          line_if_bracket <- intersect(left[bracket],ifs)
          
          #determine a condition and append it to the vector of conditions
          left_parenthesis <- str_locate_all(f[[line_if_bracket]], "\\(")
          right_parenthesis <- str_locate_all(f[[line_if_bracket]], "\\)")
          
          condition <- str_sub(f[[line_if_bracket]], (left_parenthesis[[1]][[1]]+1),
                               (right_parenthesis[[1]][[length(right_parenthesis[[1]])]]-1))
          conditions <- c(conditions, paste(condition, "&"))

          #if left[bracket[assignment]] is in if_elses
          } else if (left[bracket]  %in% if_elses ) {
            
            line_if_else_bracket <- intersect(left[bracket],if_elses)
            
            #determine a condition and append it to the vector of conditions
            left_parenthesis <- str_locate_all(f[[line_if_else_bracket]], "\\(")
            right_parenthesis <- str_locate_all(f[[line_if_else_bracket]], "\\)")
            
            condition <- str_sub(f[[line_if_else_bracket]], (left_parenthesis[[1]][[1]]+1),
                                 (right_parenthesis[[1]][[length(right_parenthesis[[1]])]]-1))
            conditions <- c(conditions, paste(condition, "&"))
            
          #if left[bracket[assignment]] is in elses
          } else if (left[bracket] %in% elses){
            
            line_else_bracket <- intersect(left[bracket],elses)
            
            #assume the function contains if_elses
            #and assign FALSE to the penultimate conditional statement (assuming it is if_else)
            first_if <- FALSE
            
            #append conditions to the vector until first_if == TRUE
            while (first_if == FALSE){

              close_if_bracket <- (line_else_bracket-1)
              open_if_bracket <- bracket_pairs[which(bracket_pairs == close_if_bracket)-1]
              
              #if length open_if_bracket == 0, abort and return the assignment with the conditions set so far
              if(length(open_if_bracket) == 0) { break }
              
              #if length open_if_bracket == 0:
              #determine a condition and append it to the vector of conditions
              left_parenthesis <- str_locate_all(f[[open_if_bracket]], "\\(")
              right_parenthesis <- str_locate_all(f[[open_if_bracket]], "\\)")
              
              condition <- str_sub(f[[open_if_bracket]], (left_parenthesis[[1]][[1]]+1),
                                   (right_parenthesis[[1]][[length(right_parenthesis[[1]])]]-1))
              conditions <- c(conditions, paste("not", condition, "&"))
          
              line_else_bracket <- open_if_bracket
              
              #check if first_if equals if (or else_if) 
              first_if <- ifelse(line_else_bracket %in% ifs, TRUE, FALSE)
            }
          }
      }
    }
    
    #create (un)conditional assignment as an object of class assignment
    a <- capture.output(cat(str_pad(f[[assignments[[i]]]], width = 60, side = "right"), "|||", rev(conditions), "TRUE\n"))
    assign_one <- assignment(a)
    
    #append an object of class assignment to a list assignment_base
    assignment_base <- append(assignment_base, assign_one)
  }
  
  #create and return an object of class assignmentList
  assignment_inf <- assignmentList(assignment_base)
  return(assignment_inf)
}

In [5]:
#main function
getFlow <- function(func){
  
  f <- deparse(body(func))
  
  #the lines where it appears: "<-", "if", "else", "else if"
  assignments <- str_which(f, "<-")
  ifs <- str_which(f, "if")
  elses <- str_which(f, "else")
  if_elses <- str_which(f, "else if")

  #if if_elses exist, determine separable parts
  if (length(if_elses) != 0){
    ifs <- setdiff(ifs,if_elses)
    elses <- setdiff(elses,if_elses)
  }
  
  #the lines where it appears: "{", "}"
  left_brackets <- str_which(f, "\\{")
  left_brackets <- left_brackets[-1]
  right_brackets <- str_which(f, "\\}")
  right_brackets <- right_brackets[-length(right_brackets)]
  
  #if lack of assignments, return "No assignments"
  if (length(assignments) == 0){
    return("No assignments")
  }
  
  #if lack of ifs, return unconditional assignments as an object of class assignmentList
  if (length(left_brackets) == 0){
    assignment_base <- list() 
    for (i in 1:length(assignments)){
      
      #create unconditional assignment as an object of class assignment
      a <- capture.output(cat(str_pad(f[[assignments[[i]]]], width = 60, side = "right"), "||| TRUE\n"))
      assign_one <- assignment(a) 
      
      #and append an object od class assignment to the list assignment_base
      assignment_base <- append(assignment_base, assign_one) 
    }
    
    #create and return an object of class assignmentList
    assignment_inf <- assignmentList(assignment_base)
    return(assignment_inf)
    
  } else {
  
  #if ifs exist:
  #determine pairs for left "{" and right "}"
  bracket_pairs <- make_pairs(left_brackets, right_brackets)
  
  #check which assignments belong to which intervals
  assignments_in_intervals <- to_which_interval_belongs(bracket_pairs, assignments)
  
  #determine the conditions of assignments and return an object of class assignmentList
  assignment_inf <- check_assigments_conditions(assignments_in_intervals, assignments, ifs, elses, if_elses, f, bracket_pairs)
  return(assignment_inf)
  }
}

## Examples

In [6]:
f <- function(a) {
  x <- a + rnorm(1)
  x
}

fFlow <- getFlow(f)
fFlow

[1]     x <- a + rnorm(1)                                        ||| TRUE

In [7]:
f <- function(a, b) {
  b <- abs(b)
  x <- a ^ b
  x
}

fFlow <- getFlow(f)
fFlow

[1]     b <- abs(b)                                              ||| TRUE
[1]     x <- a^b                                                 ||| TRUE

In [8]:
f <- function(z, b) {
  if( z <= 0) {
    stop("Only positive values of z are allowed.", call. = FALSE)
  } else {
    x <- z ^ b
  }
  x
}

fFlow <- getFlow(f)
fFlow

[1]         x <- z^b                                             ||| not z <= 0 & TRUE

In [9]:
f <- function(a, b) {
  x <- rnorm(1)
  if(a < 0)
  {
    a <- abs(a)
    if(b == 2L)
    {
      y <- a ^ 2
      y <- y + x
    }
  } else
  {
    y <- a ^ b
    y <- y + x
  }
  y
}

fFlow <- getFlow(f)
fFlow

[1]     x <- rnorm(1)                                            ||| TRUE
[1]         a <- abs(a)                                          ||| a < 0 & TRUE
[1]             y <- a^2                                         ||| a < 0 & b == 2L & TRUE
[1]             y <- y + x                                       ||| a < 0 & b == 2L & TRUE
[1]         y <- a^b                                             ||| not a < 0 & TRUE
[1]         y <- y + x                                           ||| not a < 0 & TRUE

In [10]:
f <- function(a, b) {
  q <- c("Asia", "Japan", "Tokyo", "Ginza")
  if (q[1] == "Europe") {
    y <- "Europe"
  } else if (q[1] == "Asia") {
    y <- "Asia"
    if (q[2] == "China") {
      yy <- "Asia China"
    } else if (q[2] == "Japan") {
      yy <- "Asia Japan"
      if (q[3] == "Tokyo") {
        yyy <- "Asia Japan Tokyo"
        if (q[4] == "Ginza") {
          yyyy <-  "Asia Japan Tokyo Ginza"
        } else if (q[4] == "Zoo") {
          yyyy <-  "Asia Japan Tokyo Zoo"
        } else {
          yyyy <- "No more attractions"
        }
      } else if (q[3] == "Osaka"){
        yyy <- "Asia Japan Osaka"
      } else {
        yyy <- "No more cities"
      }
    } else {
      yy <- "No more countries"
    }
  } else if (q[1] == "Africa") {
    y <- "Africa"
  } else {
    y <- "No more continents"
  }
}

fFlow <- getFlow(f)
fFlow

[1]     q <- c("Asia", "Japan", "Tokyo", "Ginza")                ||| TRUE
[1]         y <- "Europe"                                        ||| q[1] == "Europe" & TRUE
[1]         y <- "Asia"                                          ||| q[1] == "Asia" & TRUE
[1]             yy <- "Asia China"                               ||| q[1] == "Asia" & q[2] == "China" & TRUE
[1]             yy <- "Asia Japan"                               ||| q[1] == "Asia" & q[2] == "Japan" & TRUE
[1]                 yyy <- "Asia Japan Tokyo"                    ||| q[1] == "Asia" & q[2] == "Japan" & q[3] == "Tokyo" & TRUE
[1]                   yyyy <- "Asia Japan Tokyo Ginza"           ||| q[1] == "Asia" & q[2] == "Japan" & q[3] == "Tokyo" & q[4] == "Ginza" & TRUE
[1]                   yyyy <- "Asia Japan Tokyo Zoo"             ||| q[1] == "Asia" & q[2] == "Japan" & q[3] == "Tokyo" & q[4] == "Zoo" & TRUE
[1]                   yyyy <- "No more attractions"              ||| q[1] == "Asia" & q[2] == "Japan" & q[3] 

In [11]:
f <- function(a, b){
  
  if (2==1) {
    print(1)
  } else {
    print(2)
  }
}

fFlow <- getFlow(f)
fFlow

[1] "No assignments"